# Import Sources
This example shows how to import voltage and current sources. In this example, we are going to

- Download an example board
- Create a configuration file
  - Add a voltage source between two nets
  - Add a current source between two pins
  - Add a current source between two pin groups
  - Add a current source between two coordinates
  - Add a current source to the nearest pin
  - Add distributed sources
- Import the configuration file

## Perform imports and define constants

Perform required imports.

In [1]:
import json
import toml
from pathlib import Path
import tempfile

from ansys.aedt.core.examples.downloads import download_file
from pyedb import Edb

Define constants.

In [2]:
AEDT_VERSION = "2025.2"
NG_MODE = False

Download the example PCB data.

In [3]:
temp_folder = tempfile.TemporaryDirectory(suffix=".ansys")
file_edb = download_file(source="edb/ANSYS-HSD_V1.aedb", local_path=temp_folder.name)

## Load example layout

In [4]:
edbapp = Edb(file_edb, edbversion=AEDT_VERSION)

C:\actions-runner\_work\pyaedt-examples\pyaedt-examples\.venv\lib\site-packages\pyedb\misc\decorators.py:55: UserWarning: Argument `edbversion` is deprecated for method `Edb`; use `version` instead.
  warnings.warn(
C:\actions-runner\_work\pyaedt-examples\pyaedt-examples\.venv\lib\site-packages\pyedb\generic\design_types.py:301: UserWarning: Your ANSYS AEDT version is eligible to gRPC version.You might consider switching to that version for better user experience.For more information please check this link: https://edb.docs.pyansys.com/version/dev/grpc_api/index.html
  warnings.warn(GRPC_GENERAL_WARNING, UserWarning)


PyEDB INFO: Star initializing Edb 10:30:10.558284


PyEDB INFO: Edb version 2025.2


PyEDB INFO: Logger is initialized. Log file is saved to C:\Users\ansys\AppData\Local\Temp\pyedb_ansys.log.


PyEDB INFO: legacy v0.67.1


...

PyEDB INFO: Database ANSYS-HSD_V1.aedb Opened in 2025.2


PyEDB INFO: Cell main Opened


PyEDB INFO: Builder was initialized.


PyEDB INFO: open_edb completed in 9.0567 seconds.


PyEDB INFO: EDB initialization completed in 9.1210 seconds.


## Create an empty dictionary to host all configurations

In [5]:
cfg = dict()

## Add a voltage source between two nets

Keywords

- **name**. Name of the voltage source.
- **Reference_designator**. Reference designator of the component.
- **type**. Type of the source. Supported types are 'voltage', 'current'
- **impedance**. Impedance of the port. Default is 5e7 Ohm for current sources, 1e-6 Ohm for voltage sources.
- **positive_terminal**. Supported types are 'net', 'pin', 'pin_group', 'coordinates'
  - **contact_radius**. Optional. Set circular equipotential region.
  - **inline**. Optional. When True, contact points are placed in a row.
  - **num_of_contact**. Optional. Number of contact points. Default is 1. Applicable only when inline is True.
- **negative_terminal**. Supported types are 'net', 'pin', 'pin_group', 'coordinates'
- **equipotential**. Set equipotential region on pins when True.

In [6]:
voltage_source = {
    "name": "V_SOURCE_5V",
    "reference_designator": "U4",
    "type": "voltage",
    "magnitude": 1,
    "positive_terminal": {"net": "5V", "contact_radius": "1mm"},
    "negative_terminal": {"net": "GND", "contact_radius": "1mm"},
    "equipotential": True,
}

## Add a current source between two pins

In [7]:
current_source_1 = {
    "name": "I_CURRENT_1A",
    "reference_designator": "J5",
    "type": "current",
    "magnitude": 10,
    "positive_terminal": {"pin": "15"},
    "negative_terminal": {"pin": "14"},
}

## Add a current source between two pin groups

In [8]:
pin_groups = [
    {"name": "IC2_5V", "reference_designator": "IC2", "pins": ["8"]},
    {"name": "IC2_GND", "reference_designator": "IC2", "net": "GND"},
]

In [9]:
current_source_2 = {
    "name": "CURRENT_SOURCE_2",
    "type": "current",
    "positive_terminal": {"pin_group": "IC2_5V"},
    "negative_terminal": {"pin_group": "IC2_GND"},
}

## Add a current source between two coordinates

Keywords

- **layer**. Layer on which the terminal is placed
- **point**. XY coordinate the terminal is placed
- **net**. Name of the net the terminal is placed on

In [10]:
current_source_3 = {
    "name": "CURRENT_SOURCE_3",
    "type": "current",
    "equipotential": True,
    "positive_terminal": {"coordinates": {"layer": "1_Top", "point": ["116mm", "41mm"], "net": "5V"}},
    "negative_terminal": {"coordinates": {"layer": "Inner1(GND1)", "point": ["116mm", "41mm"], "net": "GND"}},
}

## Add a current source reference to the nearest pin

Keywords

- **reference_net**. Name of the reference net
- **search_radius**. Reference pin search radius in meter

In [11]:
current_source_4 = {
    "name": "CURRENT_SOURCE_4",
    "reference_designator": "J5",
    "type": "current",
    "positive_terminal": {"pin": "16"},
    "negative_terminal": {"nearest_pin": {"reference_net": "GND", "search_radius": 5e-3}},
}

## Add distributed current sources

Keywords

- **distributed**. Whether to create distributed sources. When set to True, ports are created per pin

In [12]:
sources_distributed = {
    "name": "DISTRIBUTED",
    "reference_designator": "U2",
    "type": "current",
    "distributed": True,
    "positive_terminal": {"net": "5V"},
    "negative_terminal": {"net": "GND"},
}

## Add setups in configuration

In [13]:
cfg["pin_groups"] = pin_groups
cfg["sources"] = [
    voltage_source,
    current_source_1,
    current_source_2,
    current_source_3,
    current_source_4,
    sources_distributed,
]

## Write configuration into as JSON file

In [14]:
file_json = Path(temp_folder.name) / "edb_configuration.json"
with open(file_json, "w") as f:
    json.dump(cfg, f, indent=4, ensure_ascii=False)

Equivalent toml file looks like below 

In [15]:
toml_string = toml.dumps(cfg)
print(toml_string)

[[pin_groups]]
name = "IC2_5V"
reference_designator = "IC2"
pins = [ "8",]

[[pin_groups]]
name = "IC2_GND"
reference_designator = "IC2"
net = "GND"

[[sources]]
name = "V_SOURCE_5V"
reference_designator = "U4"
type = "voltage"
magnitude = 1
equipotential = true

[sources.positive_terminal]
net = "5V"
contact_radius = "1mm"
[sources.negative_terminal]
net = "GND"
contact_radius = "1mm"
[[sources]]
name = "I_CURRENT_1A"
reference_designator = "J5"
type = "current"
magnitude = 10

[sources.positive_terminal]
pin = "15"
[sources.negative_terminal]
pin = "14"
[[sources]]
name = "CURRENT_SOURCE_2"
type = "current"

[sources.positive_terminal]
pin_group = "IC2_5V"
[sources.negative_terminal]
pin_group = "IC2_GND"
[[sources]]
name = "CURRENT_SOURCE_3"
type = "current"
equipotential = true

[sources.positive_terminal.coordinates]
layer = "1_Top"
point = [ "116mm", "41mm",]
net = "5V"
[sources.negative_terminal.coordinates]
layer = "Inner1(GND1)"
point = [ "116mm", "41mm",]
net = "GND"
[[source

## Import configuration into example layout

In [16]:
edbapp.configuration.load(config_file=file_json)
edbapp.configuration.run()

PyEDB INFO: Updating nets finished. Time lapse 0:00:00


PyEDB INFO: Updating components finished. Time lapse 0:00:00


PyEDB INFO: Creating pin groups finished. Time lapse 0:00:00.015669


PyEDB INFO: Placing sources finished. Time lapse 0:00:00.553225


PyEDB INFO: Applying materials finished. Time lapse 0:00:00


...

PyEDB INFO: Placing ports finished. Time lapse 0:00:00


PyEDB INFO: Placing terminals completed in 0.0000 seconds.


PyEDB INFO: Placing probes finished. Time lapse 0:00:00


PyEDB INFO: Applying operations completed in 0.0000 seconds.


True

## Review

In [17]:
print(edbapp.siwave.sources)

{'V_SOURCE_5V': <pyedb.dotnet.database.edb_data.ports.ExcitationSources object at 0x00000159952ACB20>, 'I_CURRENT_1A': <pyedb.dotnet.database.edb_data.ports.ExcitationSources object at 0x00000159952ADA80>, 'CURRENT_SOURCE_2': <pyedb.dotnet.database.edb_data.ports.ExcitationSources object at 0x00000159952ADCC0>, 'CURRENT_SOURCE_3': <pyedb.dotnet.database.edb_data.ports.ExcitationSources object at 0x00000159952AF7C0>, 'CURRENT_SOURCE_4': <pyedb.dotnet.database.edb_data.ports.ExcitationSources object at 0x00000159952AEDD0>, 'U2_5V_39': <pyedb.dotnet.database.edb_data.ports.ExcitationSources object at 0x00000159952ADEA0>, 'U2_5V_40': <pyedb.dotnet.database.edb_data.ports.ExcitationSources object at 0x00000159952AE140>, 'U2_5V_41': <pyedb.dotnet.database.edb_data.ports.ExcitationSources object at 0x0000015995335240>, 'U2_5V_42': <pyedb.dotnet.database.edb_data.ports.ExcitationSources object at 0x00000159953345B0>, 'U2_5V_43': <pyedb.dotnet.database.edb_data.ports.ExcitationSources object at

## Save and close Edb
The temporary folder will be deleted once the execution of this script is finished. Replace **edbapp.save()** with
**edbapp.save_as("C:/example.aedb")** to keep the example project.

In [18]:
edbapp.save()
edbapp.close()

PyEDB INFO: Save Edb file completed in 0.1225 seconds.


PyEDB INFO: Close Edb file completed in 0.0636 seconds.


True